In [104]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [116]:
import tensorflow.keras as keras
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import tensorflow.keras.callbacks as callbacks
from tensorflow.keras.utils import plot_model 

from nltk import sent_tokenize
from tqdm import tqdm 
import numpy as np
import re
import string
import pandas as pd

In [5]:
path_to_file = 'data/40000-Egyptian-tweets.xlsx'

In [6]:
df = pd.read_excel(path_to_file)

In [8]:
df.drop(['label'], axis=1, inplace=True)

In [9]:
df.head()

,review
0,اكبر خطا ترتكبه ان تعامل الناس باخلاقك انت مش ...
1,دائما اكره اخر ليله في كل مكان .
2,يارب اللى يسرق تويتاتى يدخل النار .
3,الاسراف فى تناول القهوة يسبب الوفاه .
4,انا اتبهدلت من التراب النهارده. حاجة تقرف .


In [16]:
df.to_numpy()[0]

array(['اكبر خطا ترتكبه ان تعامل الناس باخلاقك انت مش باخلاقهم هما  .  '],
      dtype=object)

In [22]:
corpus = [sent[0] for sent in df.to_numpy()]
corpus[:2]

['اكبر خطا ترتكبه ان تعامل الناس باخلاقك انت مش باخلاقهم هما  .  ',
 'دائما اكره اخر ليله في كل مكان   .  ']

### Preprocessing 

In [23]:
punctuation_string = '!"\\\\#\\$%\\&\\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~\d'

In [105]:
import pyarabic.araby as araby
from pyarabic.araby import strip_tashkeel, strip_tatweel, strip_tashkeel, is_arabicrange

In [179]:
def split_al(word):
    if word.startswith('ال'):
        return word[:2], word[2:]
    else: 
        return word

    
split_al('الراحه')

('ال', 'راحه')

In [185]:
def split_al_sent(sent):
    ww = []
    for word in sent.split():
        out = split_al(word)
        if type(out) is tuple:
            for w in out:
                ww.append(w)
        else:
            ww.append(word)
    return ' '.join(w for w in ww)

In [186]:
split_al_sent('الراحه ببساطة فى القرآن فى الصلاة فى الدعاء فى...')

'ال راحه ببساطة فى ال قرآن فى ال صلاة فى ال دعاء فى...'

In [188]:
conditions = [is_arabicrange]
morphs = [strip_tashkeel, strip_tatweel]

In [189]:
def preprocess(corpus):
    sentences = sent_tokenize(' '.join(w for w in corpus))
#     sentences = [sent.lower() for sent in sentences]
#     sentences = [re.sub('<\w+>', '', sent) for sent in sentences]
    sentences = [sent.decode() for sent in 
                 tf.strings.regex_replace(sentences,f'[{punctuation_string}]', '').numpy()]
    sentences = [split_al_sent(sent) for sent in sentences]
    sentences = [araby.normalize_hamza(sent, method='tasheel') for sent in sentences]
#     tokens = [word_tokenize(sent) for sent in tqdm(sentences)]
    tokens = [araby.tokenize(sent, conditions=conditions, morphs=morphs) for sent in tqdm(sentences)]
    # return tokenized sentences
    return tokens

In [195]:
tokens_list = preprocess(corpus)

100%|██████████| 44996/44996 [00:01<00:00, 40167.37it/s]


In [196]:
len(tokens_list)

44996

In [197]:
print(tokens_list[0])

['اكبر', 'خطا', 'ترتكبه', 'ان', 'تعامل', 'ال', 'ناس', 'باخلاقك', 'انت', 'مش', 'باخلاقهم', 'هما']


In [198]:
def build_vocabulary(tokens_list):
    stoi = {}
    itos = {}
    idx = 0
    
    stoi['<pad>'] = idx
    itos[idx] = '<pad>'
    idx += 1
    
    for tokens in tqdm(tokens_list):
        for word in tokens:
            if word not in stoi:
                stoi[word] = idx
                itos[idx] = word
                idx += 1
    return stoi, itos

In [199]:
# remove '
tokens_list
stoi, itos = build_vocabulary(tokens_list)

100%|██████████| 44996/44996 [00:00<00:00, 365069.90it/s]


In [200]:
numericalized = [[stoi[w] for w in sent] for sent in tokens_list]

In [201]:
vocab_size = len(stoi)
embed_size = 300
window_size = 2

print(f"Vocab Size  : {vocab_size}")
print(f"Vocab Sample: {list(stoi.items())[:5]}")

Vocab Size  : 49454
Vocab Sample: [('<pad>', 0), ('اكبر', 1), ('خطا', 2), ('ترتكبه', 3), ('ان', 4)]


In [202]:
def generate_context_target_pairs(corpus=numericalized, window_size=2, vocab_size=vocab_size):
    
    context_length = window_size * 2
    
    for sentence in corpus:
        sentence_length = len(sentence)
        
        for index, word in enumerate(sentence):
            context_words = []
            label_word = []
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([sentence[i] for i in range(start, end)
                                 if 0 <= i < sentence_length and i != index])
            label_word.append(word)
            
            x = pad_sequences(context_words, maxlen=context_length)
            y = to_categorical(label_word, vocab_size)
            yield(x[0], y[0])

In [204]:
i = 0
for x, y in generate_context_target_pairs(numericalized, 2, vocab_size):
    if 0 not in x:
        if i == 10: break
        i+=1

        print(f'Target: {itos[np.argmax(y)]}, Context: {[itos[w] for w in x]}')

Target: ترتكبه, Context: ['اكبر', 'خطا', 'ان', 'تعامل']
Target: ان, Context: ['خطا', 'ترتكبه', 'تعامل', 'ال']
Target: تعامل, Context: ['ترتكبه', 'ان', 'ال', 'ناس']
Target: ال, Context: ['ان', 'تعامل', 'ناس', 'باخلاقك']
Target: ناس, Context: ['تعامل', 'ال', 'باخلاقك', 'انت']
Target: باخلاقك, Context: ['ال', 'ناس', 'انت', 'مش']
Target: انت, Context: ['ناس', 'باخلاقك', 'مش', 'باخلاقهم']
Target: مش, Context: ['باخلاقك', 'انت', 'باخلاقهم', 'هما']
Target: اخر, Context: ['دايما', 'اكره', 'ليله', 'في']
Target: ليله, Context: ['اكره', 'اخر', 'في', 'كل']


In [205]:
output_shapes = ([None, window_size*2],[None, vocab_size])
output_shapes

([None, 4], [None, 49454])

In [206]:
output_shapes = ([window_size*2],[vocab_size])
dataset = tf.data.Dataset.from_generator(generate_context_target_pairs, (tf.int32, tf.int32), output_shapes=output_shapes)
dataset = dataset.batch(1000)

In [207]:
for batch, (x, y) in enumerate(dataset):
    break
    #print("batch: ", batch)
print(x.shape, y.shape)

(1000, 4) (1000, 49454)


In [155]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2),
    Lambda(lambda x: keras.backend.mean(x, axis=1), output_shape=(embed_size)),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [156]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 300)            174600    
_________________________________________________________________
lambda_1 (Lambda)            (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 582)               175182    
Total params: 349,782
Trainable params: 349,782
Non-trainable params: 0
_________________________________________________________________


In [157]:
plot_model(model,show_shapes=True, show_layer_names=True, rankdir='TB', expand_nested=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [158]:
# google_drive_path = '/content/drive/MyDrive/weights/cbow/now_1000.h5'
google_drive_path = 'models/test_ar.h5'

In [159]:
early_stopping_cb = callbacks.EarlyStopping(monitor='loss', patience=4, verbose=True)
checkpoint_cb = callbacks.ModelCheckpoint(google_drive_path, monitor='loss', verbose=True, save_best_only=True)
tensorboard_callback = callbacks.TensorBoard(log_dir="logs")

callbacks_list = [early_stopping_cb, checkpoint_cb, tensorboard_callback]

history = model.fit(dataset, epochs=1, callbacks=callbacks_list)

1/1 [==============================] - 13s 13s/step - loss: 6.3656 - accuracy: 0.0013

Epoch 00001: loss improved from inf to 6.36562, saving model to models/test_ar.h5
